# Name Gender Identifier

## 1. Building a feature extractor

An idea is to use the last letter of the name to predict the gender. For instance, names ending in *a*, *e* and *i* are likely to be female, while names ending in *k*, *o*, *r*, *s* and *t* are likely to be male.

In [1]:
# Feature extractor
def gender_features(word):
    return {'last_letter': word[-1]}

gender_features('John')

{'last_letter': 'n'}

The returned dictionary is known as a **feature set**.

## 2. Exploring the `names` corpus

In [2]:
from nltk.corpus import names

names.readme().replace('\n', ' ')

'Names Corpus, Version 1.3 (1994-03-29) Copyright (C) 1991 Mark Kantrowitz Additions by Bill Ross  This corpus contains 5001 female names and 2943 male names, sorted alphabetically, one per line.  You may use the lists of names for any purpose, so long as credit is given in any published work. You may also redistribute the list if you provide the recipients with a copy of this README file. The lists are not in the public domain (I retain the copyright on the lists) but are freely redistributable.  If you have any additions to the lists of names, I would appreciate receiving them.  Mark Kantrowitz <mkant+@cs.cmu.edu> http://www-2.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/'

In [3]:
names.fileids()

['female.txt', 'male.txt']

In [4]:
names.words('female.txt')[:5]

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi']

## 3. Building the classifier

We need to prepare a list of examples and corresponding class labels.

In [5]:
labeled_names = ([(name, 'female') for name in names.words('female.txt')] + [(name, 'male') for name in names.words('male.txt')])
labeled_names[:5]

[('Abagael', 'female'),
 ('Abagail', 'female'),
 ('Abbe', 'female'),
 ('Abbey', 'female'),
 ('Abbi', 'female')]

In [6]:
import random
random.shuffle(labeled_names) # We shuffle the data so that we can split it by index into training and test data.
labeled_names[:5]

[('Patrica', 'female'),
 ('Kristy', 'female'),
 ('Kate', 'female'),
 ('Tomlin', 'male'),
 ('Amadeus', 'male')]

In [7]:
feature_sets = [(gender_features(n), gender) for (n, gender) in labeled_names]
feature_sets[:5]

[({'last_letter': 'a'}, 'female'),
 ({'last_letter': 'y'}, 'female'),
 ({'last_letter': 'e'}, 'female'),
 ({'last_letter': 'n'}, 'male'),
 ({'last_letter': 's'}, 'male')]

In [8]:
len(feature_sets)

7944

In [9]:
from nltk import NaiveBayesClassifier

# We split the data into a training (80%) and test (20%) set:
TRAIN_SET_SIZE = round(len(feature_sets) * .8)
train_set, test_set = feature_sets[:TRAIN_SET_SIZE], feature_sets[TRAIN_SET_SIZE:]

# We also get the names in the test set, to be used later:
test_names = labeled_names[TRAIN_SET_SIZE:]

classifier = NaiveBayesClassifier.train(train_set)

# When working with large corpora, constructing a single list that contains the features of every instance can use up a large amount of memory. In these cases, use the function nltk.classify.apply_features, which returns an object that acts like a list but does not store all the feature sets in memory:
# from nltk.classify import apply_features
# train_names, test_names = labeled_names[:round(len(feature_sets) * .8)], labeled_names[round(len(feature_sets) * .8):]
# train_set = apply_features(gender_features, labeled_names[500:])
# test_set = apply_features(gender_features, labeled_names[:500])

In [10]:
classifier.show_most_informative_features(10) # Prints likelihood ratios for most informative features

Most Informative Features
             last_letter = 'a'            female : male   =     35.9 : 1.0
             last_letter = 'k'              male : female =     29.6 : 1.0
             last_letter = 'f'              male : female =     13.1 : 1.0
             last_letter = 'p'              male : female =     11.1 : 1.0
             last_letter = 'o'              male : female =      9.0 : 1.0
             last_letter = 'd'              male : female =      8.8 : 1.0
             last_letter = 'm'              male : female =      7.5 : 1.0
             last_letter = 'v'              male : female =      7.1 : 1.0
             last_letter = 'r'              male : female =      6.4 : 1.0
             last_letter = 'g'              male : female =      5.9 : 1.0


## 4. Testing the classifier

In [11]:
classifier.labels()

['female', 'male']

In [12]:
from nltk.classify import accuracy

round(accuracy(classifier, test_set), 2)

0.77

In [13]:
classifier.classify(gender_features('Aphrodite'))

'female'

In [14]:
classifier.classify(gender_features('Zeus'))

'male'

## 5. Building a classifier with more features

In [15]:
def gender_features_2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

gender_features_2('John')

{'first_letter': 'j',
 'last_letter': 'n',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [16]:
feature_sets_2 = [(gender_features_2(n), gender) for (n, gender) in labeled_names]
feature_sets_2[0]

({'first_letter': 'p',
  'last_letter': 'a',
  'count(a)': 2,
  'has(a)': True,
  'count(b)': 0,
  'has(b)': False,
  'count(c)': 1,
  'has(c)': True,
  'count(d)': 0,
  'has(d)': False,
  'count(e)': 0,
  'has(e)': False,
  'count(f)': 0,
  'has(f)': False,
  'count(g)': 0,
  'has(g)': False,
  'count(h)': 0,
  'has(h)': False,
  'count(i)': 1,
  'has(i)': True,
  'count(j)': 0,
  'has(j)': False,
  'count(k)': 0,
  'has(k)': False,
  'count(l)': 0,
  'has(l)': False,
  'count(m)': 0,
  'has(m)': False,
  'count(n)': 0,
  'has(n)': False,
  'count(o)': 0,
  'has(o)': False,
  'count(p)': 1,
  'has(p)': True,
  'count(q)': 0,
  'has(q)': False,
  'count(r)': 1,
  'has(r)': True,
  'count(s)': 0,
  'has(s)': False,
  'count(t)': 1,
  'has(t)': True,
  'count(u)': 0,
  'has(u)': False,
  'count(v)': 0,
  'has(v)': False,
  'count(w)': 0,
  'has(w)': False,
  'count(x)': 0,
  'has(x)': False,
  'count(y)': 0,
  'has(y)': False,
  'count(z)': 0,
  'has(z)': False},
 'female')

In [17]:
train_set_2, test_set_2 = feature_sets_2[:TRAIN_SET_SIZE], feature_sets_2[TRAIN_SET_SIZE:]
classifier_2 = NaiveBayesClassifier.train(train_set_2)
round(accuracy(classifier_2, test_set_2), 2)

0.77

We would have expected that having too many specific features on a small dataset would lead to overfitting, but it seems the classifier was good at avoiding that since its performance is slightly better.

In [18]:
classifier_2.show_most_informative_features(15)

Most Informative Features
             last_letter = 'a'            female : male   =     35.9 : 1.0
             last_letter = 'k'              male : female =     29.6 : 1.0
             last_letter = 'f'              male : female =     13.1 : 1.0
             last_letter = 'p'              male : female =     11.1 : 1.0
             last_letter = 'o'              male : female =      9.0 : 1.0
             last_letter = 'd'              male : female =      8.8 : 1.0
                count(v) = 2              female : male   =      7.7 : 1.0
             last_letter = 'm'              male : female =      7.5 : 1.0
             last_letter = 'v'              male : female =      7.1 : 1.0
                count(a) = 3              female : male   =      6.6 : 1.0
             last_letter = 'r'              male : female =      6.4 : 1.0
             last_letter = 'g'              male : female =      5.9 : 1.0
             last_letter = 'w'              male : female =      5.0 : 1.0

Indeed, it seems the classifier is mainly using the last letter, along with some other features that happen to improve the accuracy.

## 6. Comparing the two classifiers using `nltk.metrics`

Before we start, here's a useful function for comparing strings:

In [19]:
from nltk.metrics import edit_distance

edit_distance("John", "Joan")

1

The NLTK metrics module provides functions for calculating metrics beyond mere accuracy. But in order to do so, we need to build 2 sets for each classification label: a reference set of correct values, and a test set of observed values.

In [20]:
import collections

# Classifier 1
ref_sets = collections.defaultdict(set) # For what this is: https://stackoverflow.com/questions/5900578/how-does-collections-defaultdict-work
test_sets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    ref_sets[label].add(i)
    observed = classifier.classify(feats)
    test_sets[observed].add(i)

# Classifier 2
ref_sets_2 = collections.defaultdict(set)
test_sets_2 = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set_2):
    ref_sets_2[label].add(i)
    observed = classifier_2.classify(feats)
    test_sets_2[observed].add(i)

In [21]:
ref_sets

defaultdict(set,
            {'male': {0,
              1,
              8,
              11,
              17,
              27,
              28,
              36,
              38,
              45,
              49,
              57,
              59,
              60,
              63,
              67,
              68,
              70,
              72,
              89,
              91,
              94,
              98,
              102,
              103,
              104,
              109,
              111,
              118,
              121,
              124,
              125,
              130,
              132,
              136,
              138,
              145,
              146,
              148,
              151,
              152,
              153,
              156,
              158,
              159,
              162,
              163,
              173,
              175,
              176,
              182,
              183,
             

In [22]:
test_sets

defaultdict(set,
            {'male': {0,
              1,
              6,
              7,
              8,
              9,
              10,
              11,
              19,
              24,
              25,
              26,
              27,
              28,
              43,
              45,
              46,
              51,
              57,
              60,
              62,
              65,
              68,
              70,
              72,
              75,
              78,
              79,
              86,
              89,
              94,
              98,
              100,
              102,
              103,
              106,
              107,
              109,
              110,
              111,
              113,
              118,
              120,
              126,
              130,
              136,
              139,
              141,
              143,
              145,
              146,
              151,
              152,
      

In [23]:
from nltk.metrics.scores import (precision, recall, f_measure)

# We can proceed to print the metrics for each classifier. However, we cannot get the accuracy in this manner because nltk.metrics.scores.accuracy(reference, test) works by comparing test[i] == reference[i] and our reference and test are not formatted in a way that allows for this. It's the same for the confusion matrix.
args = (
    round(precision(ref_sets['female'], test_sets['female']), 2),
    round(precision(ref_sets['male'], test_sets['male']), 2),
    round(recall(ref_sets['female'], test_sets['female']), 2),
    round(recall(ref_sets['male'], test_sets['male']), 2),
    round(f_measure(ref_sets['female'], test_sets['female']), 2),
    round(f_measure(ref_sets['male'], test_sets['male']), 2)
)

args2 = (
    round(precision(ref_sets_2['female'], test_sets_2['female']), 2),
    round(precision(ref_sets_2['male'], test_sets_2['male']), 2),
    round(recall(ref_sets_2['female'], test_sets_2['female']), 2),
    round(recall(ref_sets_2['male'], test_sets_2['male']), 2),
    round(f_measure(ref_sets_2['female'], test_sets_2['female']), 2),
    round(f_measure(ref_sets_2['male'], test_sets_2['male']), 2)
)

print('''
CLASSIFIER 1
------------
Female precision: {0}
Male precision: {1}
Female recall: {2}
Male recall: {3}
Female F1 score: {4}
Male F1 score: {5}

CLASSIFIER 2
------------
Female precision: {6}
Male precision: {7}
Female recall: {8}
Male recall: {9}
Female F1 score: {10}
Male F1 score: {11}
'''.format(*args, *args2))


CLASSIFIER 1
------------
Female precision: 0.82
Male precision: 0.69
Female recall: 0.83
Male recall: 0.68
Female F1 score: 0.82
Male F1 score: 0.69

CLASSIFIER 2
------------
Female precision: 0.83
Male precision: 0.67
Female recall: 0.8
Male recall: 0.71
Female F1 score: 0.81
Male F1 score: 0.69



## 7. Error analysis

In [24]:
errors = []
for name, tag in test_names:
    guess = classifier_2.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

errors[:5]

[('female', 'male', 'Birgit'),
 ('female', 'male', 'Inez'),
 ('female', 'male', 'Cass'),
 ('female', 'male', 'Adriaens'),
 ('male', 'female', 'Marsh')]

In [25]:
for tag, guess, name in sorted(errors):
    print('Correct = {:8} guess = {:8} name = {}'.format(tag, guess, name)) # :8 creates spaces between columns.

Correct = female   guess = male     name = Adel
Correct = female   guess = male     name = Adriaens
Correct = female   guess = male     name = Adrian
Correct = female   guess = male     name = Agnes
Correct = female   guess = male     name = Allyn
Correct = female   guess = male     name = Allyson
Correct = female   guess = male     name = Anabel
Correct = female   guess = male     name = Anais
Correct = female   guess = male     name = Angil
Correct = female   guess = male     name = Annabal
Correct = female   guess = male     name = Arabel
Correct = female   guess = male     name = Bab
Correct = female   guess = male     name = Beilul
Correct = female   guess = male     name = Berget
Correct = female   guess = male     name = Beryl
Correct = female   guess = male     name = Bidget
Correct = female   guess = male     name = Bird
Correct = female   guess = male     name = Birgit
Correct = female   guess = male     name = Bliss
Correct = female   guess = male     name = Brenn
Correct = 

Looking through this list of errors, it seems that some suffixes that are more than one letter long can be indicative of name genders. For example, names ending in *yn* appear to be predominantly female, despite the fact that names ending in *n* tend to be male; and names ending in *ch* are usually male, even though names that end in *h* tend to be female.

## 8. Building a classifier with even more features

In [26]:
def gender_features_3(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["suffix1"] = name[-1].lower()
    features["suffix2"] = name[-2:].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

gender_features_3('John')

{'first_letter': 'j',
 'suffix1': 'n',
 'suffix2': 'hn',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [27]:
feature_sets_3 = [(gender_features_3(n), gender) for (n, gender) in labeled_names]
feature_sets_3[0]

({'first_letter': 'p',
  'suffix1': 'a',
  'suffix2': 'ca',
  'count(a)': 2,
  'has(a)': True,
  'count(b)': 0,
  'has(b)': False,
  'count(c)': 1,
  'has(c)': True,
  'count(d)': 0,
  'has(d)': False,
  'count(e)': 0,
  'has(e)': False,
  'count(f)': 0,
  'has(f)': False,
  'count(g)': 0,
  'has(g)': False,
  'count(h)': 0,
  'has(h)': False,
  'count(i)': 1,
  'has(i)': True,
  'count(j)': 0,
  'has(j)': False,
  'count(k)': 0,
  'has(k)': False,
  'count(l)': 0,
  'has(l)': False,
  'count(m)': 0,
  'has(m)': False,
  'count(n)': 0,
  'has(n)': False,
  'count(o)': 0,
  'has(o)': False,
  'count(p)': 1,
  'has(p)': True,
  'count(q)': 0,
  'has(q)': False,
  'count(r)': 1,
  'has(r)': True,
  'count(s)': 0,
  'has(s)': False,
  'count(t)': 1,
  'has(t)': True,
  'count(u)': 0,
  'has(u)': False,
  'count(v)': 0,
  'has(v)': False,
  'count(w)': 0,
  'has(w)': False,
  'count(x)': 0,
  'has(x)': False,
  'count(y)': 0,
  'has(y)': False,
  'count(z)': 0,
  'has(z)': False},
 'female'

In [28]:
train_set_3, test_set_3 = feature_sets_3[:TRAIN_SET_SIZE], feature_sets_3[TRAIN_SET_SIZE:]
classifier_3 = NaiveBayesClassifier.train(train_set_3)
round(accuracy(classifier_3, test_set_3), 2)

0.79

In [29]:
classifier_3.show_most_informative_features(15)

Most Informative Features
                 suffix2 = 'na'           female : male   =     82.5 : 1.0
                 suffix2 = 'la'           female : male   =     64.4 : 1.0
                 suffix2 = 'ia'           female : male   =     49.3 : 1.0
                 suffix2 = 'ta'           female : male   =     39.2 : 1.0
                 suffix2 = 'us'             male : female =     36.0 : 1.0
                 suffix1 = 'a'            female : male   =     35.9 : 1.0
                 suffix1 = 'k'              male : female =     29.6 : 1.0
                 suffix2 = 'rd'             male : female =     28.0 : 1.0
                 suffix2 = 'do'             male : female =     23.6 : 1.0
                 suffix2 = 'ra'           female : male   =     22.6 : 1.0
                 suffix2 = 'io'             male : female =     20.3 : 1.0
                 suffix2 = 'rt'             male : female =     20.0 : 1.0
                 suffix2 = 'ld'             male : female =     19.5 : 1.0

## 9. Trying to use a maximum entropy classifier

The principle of **maximum entropy** states that the probability distribution which best represents the current state of knowledge is the one with largest entropy.

The principle of maximum entropy is invoked when we have some piece(s) of information about a probability distribution, but not enough to characterize it completely—likely because we do not have the means or resources to do so. As an example, if all we know about a distribution is its average, we can imagine infinite shapes that yield a particular average. The principle of maximum entropy says that we should humbly choose the distribution that maximizes the amount of unpredictability contained in the distribution.

Taking the idea to the extreme, it wouldn’t be scientific to choose a distribution that simply yields the average value 100% of the time.

From all the models that fit our training data, the Maximum Entropy classifier selects the one which has the largest entropy. Due to the minimum assumptions that the Maximum Entropy classifier makes, it is usually used when we don’t know anything about the prior distributions and when it is unsafe to make any assumptions. Also, the maximum entropy classifier is used when we can’t assume the conditional independence of the features.

In [30]:
from nltk import MaxentClassifier

me_classifier = MaxentClassifier.train(train_set_3, max_iter=25) # max_iter has default value 100. In this example, the performance in terms of accuracy on the test set starts significantly improving beyond the previous model's at around 25 iterations.

  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.372
             2          -0.60421        0.628
             3          -0.58271        0.628
             4          -0.56296        0.634
             5          -0.54488        0.668
             6          -0.52835        0.702
             7          -0.51327        0.731
             8          -0.49950        0.751
             9          -0.48692        0.763
            10          -0.47542        0.776
            11          -0.46488        0.784
            12          -0.45521        0.789
            13          -0.44632        0.793
            14          -0.43813        0.796
            15          -0.43058        0.797
            16          -0.42359        0.800
            17          -0.41712        0.801
            18          -0.41111        0.804
            19          -0.40552        0.806
  

In [31]:
round(accuracy(me_classifier, test_set_3), 2) # The accuracies above were on the training set so this is what matters.

0.81

In [32]:
me_classifier.show_most_informative_features(10)

  -1.913 suffix2=='na' and label is 'male'
  -1.881 suffix2=='la' and label is 'male'
  -1.640 suffix2=='ia' and label is 'male'
  -1.554 suffix2=='ta' and label is 'male'
  -1.478 suffix1=='a' and label is 'male'
  -1.383 suffix2=='us' and label is 'female'
  -1.301 suffix1=='k' and label is 'female'
  -1.287 suffix2=='ra' and label is 'male'
  -1.229 suffix2=='do' and label is 'female'
  -1.190 suffix2=='rd' and label is 'female'


## 10. More classifiers

Scikit-learn (sklearn) is a popular library which features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN.

NLTK provides an API to quickly use sklearn classifiers in `nltk.classify.scikitlearn`. The other option is to import and use sklearn directly.

For an example of integrating sklearn with NLTK, you can check out [this](https://www.kaggle.com/alvations/basic-nlp-with-nltk) notebook on Kaggle. Kaggle is a great website for NLP and machine learning in general, creating an account is highly recommended.